In [1]:
from op_coreutils.logger import bind_contextvars, clear_contextvars, structlog
from op_coreutils.clickhouse.client import append_df


from op_datasets.processing.blockrange import BlockRange
from op_datasets.coretables.read import filter_to_date, read_core_tables
from op_datasets.processing.ozone import DateTask, split_block_range
from op_datasets.schemas import ONCHAIN_CORE_DATASETS, CoreDataset
from op_datasets.processing.write import write_to_sink



2024-10-10 16:28:00 [info     ] Loaded env var: CLICKHOUSE_GOLDSKY_DBT_HOST
2024-10-10 16:28:00 [info     ] Loaded env var: CLICKHOUSE_GOLDSKY_DBT_PORT
2024-10-10 16:28:00 [info     ] Loaded env var: CLICKHOUSE_GOLDSKY_DBT_USER
2024-10-10 16:28:00 [info     ] Loaded env var: CLICKHOUSE_GOLDSKY_DBT_PASSWORD
2024-10-10 16:28:00 [info     ] Loaded env var: GITHUB_ACTIONS_GCP_SERVICE_ACCOUNT


In [2]:

from op_datasets.processing.execute import split_block_range, split_dates

chain = "op"
block_range = BlockRange.from_spec("105376883:+500")
source_spec = "goldsky"
sinks_spec = "file://ozone"

dt = None
dataframes = None
for batch in split_block_range(block_range):
    for _dt, _dataframes in split_dates(batch, chain, source_spec):
        dt = _dt
        dataframes = _dataframes
        break
        

2024-10-10 16:28:05 [info     ] Initialized Clickhouse client.
2024-10-10 16:28:06 [info     ] Query success: blocks in 1.29s 2000 rows
2024-10-10 16:28:09 [info     ] Query success: transactions in 4.43s 14166 rows
2024-10-10 16:28:09 [info     ] Filtering blocks               dt=2023-06-09 max_block=105377999 min_block=105376000


In [3]:
dataframes.keys()

dict_keys(['blocks', 'transactions'])

In [4]:
dataframes["blocks"]

chain,network,chain_id,dt,timestamp,number,hash,parent_hash,nonce,sha3_uncles,logs_bloom,transactions_root,state_root,receipts_root,withdrawals_root,miner,difficulty,total_difficulty,size,base_fee_per_gas,gas_used,gas_limit,extra_data,transaction_count
str,str,i32,str,u32,i64,str,str,str,str,str,str,str,str,str,str,f64,f64,i64,i64,i64,i64,str,i64
"""op""","""mainnet""",10,"""2023-06-09""",1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x0700a7de40bc26fe8d1a9b51f5ca…","""0x0000000000000000""","""0x1dcc4de8dec75d7aab85b567b6cc…","""0x0020200802800008400000800800…","""0x07dd81167998b686d00eb025c093…","""0x3f93fa2d6754a8f6c677a4398446…","""0x3b6652be45133b033aec22f19ed3…","""""","""0x4200000000000000000000000000…",0.0,0.0,2134,53,676868,30000000,"""0x""",5
"""op""","""mainnet""",10,"""2023-06-09""",1686350779,105376001,"""0x36db4816b0aea3af6c9b3f1f4f33…","""0xf39de7bb1d732c9e4f44a936237a…","""0x0000000000000000""","""0x1dcc4de8dec75d7aab85b567b6cc…","""0x040001000a000000000020a00400…","""0x57df5daa8aff47e38f5767a9c1be…","""0xbcfd4ea29b78a130fc48d42b7e39…","""0x87d29faf35ebd5612fccbf8679c0…","""""","""0x4200000000000000000000000000…",0.0,0.0,4699,53,1149120,30000000,"""0x""",7
"""op""","""mainnet""",10,"""2023-06-09""",1686350781,105376002,"""0xe11a8093ae0e5bbb6fccf2157683…","""0x36db4816b0aea3af6c9b3f1f4f33…","""0x0000000000000000""","""0x1dcc4de8dec75d7aab85b567b6cc…","""0x068010080a000008400018800000…","""0x9fd507e7819cc1987c4a4e7f82a8…","""0x0a597b7425428e5e97aae3b80a5f…","""0xa317a4d3763fe2b9d1acaf0116a0…","""""","""0x4200000000000000000000000000…",0.0,0.0,4625,53,3368746,30000000,"""0x""",9
"""op""","""mainnet""",10,"""2023-06-09""",1686350783,105376003,"""0x95373c66c1660837bf9a977717a6…","""0xe11a8093ae0e5bbb6fccf2157683…","""0x0000000000000000""","""0x1dcc4de8dec75d7aab85b567b6cc…","""0x1600004008000000400000800000…","""0xf88050b75d582ff062230a3bce31…","""0x88d19aad1c8b07f8f92ad728b972…","""0x4bf34499818063108e0897c75121…","""""","""0x4200000000000000000000000000…",0.0,0.0,3117,53,4173936,30000000,"""0x""",7
"""op""","""mainnet""",10,"""2023-06-09""",1686350785,105376004,"""0x5aba44199a8e2231e9dfdf5d34ca…","""0x95373c66c1660837bf9a977717a6…","""0x0000000000000000""","""0x1dcc4de8dec75d7aab85b567b6cc…","""0x0260240800800002004402800840…","""0xc45d054308d31aea5039df0f6c14…","""0xbdfc94e4b98d7933c516ad7ba993…","""0x5f4989089282557c66cb381c7063…","""""","""0x4200000000000000000000000000…",0.0,0.0,3993,53,2970850,30000000,"""0x""",10
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""op""","""mainnet""",10,"""2023-06-09""",1686354767,105377995,"""0x70221a9a550a43cafcee6f983490…","""0x3a3984726bfcca8570cbfb4aed49…","""0x0000000000000000""","""0x1dcc4de8dec75d7aab85b567b6cc…","""0x060000000a000000040020a00000…","""0x675f91eb283384984e179e524c5a…","""0xc1fd212fea90b694036bb883597b…","""0x05024795289cf33398f0a823eb36…","""""","""0x4200000000000000000000000000…",0.0,0.0,5321,58,2845100,30000000,"""0x""",6
"""op""","""mainnet""",10,"""2023-06-09""",1686354769,105377996,"""0x1c11039e7313c341e9ed0b089c1d…","""0x70221a9a550a43cafcee6f983490…","""0x0000000000000000""","""0x1dcc4de8dec75d7aab85b567b6cc…","""0x040010500a000002400800800004…","""0x9d368953eec6e055f55cd777a3a8…","""0xa92575772a1881333a5575eb3a38…","""0xd0e8ba3c49d465f48bea09a0c047…","""""","""0x4200000000000000000000000000…",0.0,0.0,5167,58,1902960,30000000,"""0x""",12
"""op""","""mainnet""",10,"""2023-06-09""",1686354771,105377997,"""0xae2a16ca21762e21f874c5fa6e09…","""0x1c11039e7313c341e9ed0b089c1d…","""0x0000000000000000""","""0x1dcc4de8dec75d7aab85b567b6cc…","""0x060000402a010808400000880240…","""0xaa173d7365f15157ebc2cb86962f…","""0xd485043c0f989a2a3a18cb57660b…","""0xe323e1920b01399398d499ea2794…","""""","""0x4200000000000000000000000000…",0.0,0.0,4411,58,9648970,30000000,"""0x""",8


In [6]:
dataframes["transactions"]

dt,chain,network,chain_id,block_timestamp,block_number,block_hash,hash,nonce,transaction_index,from_address,to_address,value_64,value_lossless,gas,gas_price,input,transaction_type,max_fee_per_gas,max_priority_fee_per_gas,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_status,receipt_effective_gas_price,receipt_l1_gas_price,receipt_l1_gas_used,receipt_l1_fee,receipt_l1_fee_scalar,receipt_l1_blob_base_fee,receipt_l1_blob_base_fee_scalar,receipt_l1_base_fee_scalar
str,str,str,i32,u32,i64,str,str,i64,i64,str,str,i64,str,i64,i64,str,i32,i64,i64,i64,i64,str,i32,i64,i64,i64,i64,f64,i64,i64,i64
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x57f50c462841315a0e79a2650fd4…",140936,0,"""0xdeaddeaddeaddeaddeaddeaddead…","""0x4200000000000000000000000000…",0,"""0""",1000000,0,"""0x015d8eb900000000000000000000…",126,0,0,46913,46913,"""""",1,0,null,null,null,null,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x3e53770009212acc648e9faba440…",164300,1,"""0x593eb985684616112b09d6fb5d36…","""0x6a5ed50521eae16df75ad9dfb57a…",0,"""0""",5000000,214,"""0xa6f2ae3a""",0,0,0,217262,170349,"""""",1,214,17823396239,1884,22968226503764,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x484ed2a2299be2df85746aafd9eb…",966638,2,"""0xb8ff877ed78ba520ece21b1de784…","""0xa0cc33dd6f4819d473226257792a…",0,"""0""",173950,128,"""0x6c459a2800000000000000000000…",0,0,0,326160,108898,"""""",1,128,17823396239,6692,81583530659869,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0xe00836acfc2e456039312de9b217…",1,3,"""0x11ed66e8cd3fe0cd749d55664cd2…","""0xaaecb31961db53ea5ada85ee1bf4…",2496332147965529,"""2496332147965529""",22050,118,"""0x""",0,0,0,347160,21000,"""""",1,118,17823396239,1852,22578108006885,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x88b6192a9ec8df00f92864badf46…",11,4,"""0xefb0eb8c4ecdff8d08b8fd5aacbc…","""0x52629961f71c1c2564c5aa22372c…",0,"""0""",339474,102,"""0x76319ee000000000000000000000…",2,122,49,676868,329708,"""""",1,102,17823396239,4856,59200481901423,0.684,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-06-09""","""op""","""mainnet""",10,1686354775,105377999,"""0xea9dfd24e2c5145d214f28e8da6a…","""0x64f9d972224d75e06c28e77b1f12…",15,3,"""0xd163a98169ac9fd8a5613a8f8504…","""0xb0d502e938ed5f4df2e681fe6e41…",0,"""0""",322651,606,"""0x87b21efc00000000000000000000…",2,623,547,1912546,301739,"""""",1,606,17849053713,2624,32035767188951,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686354775,105377999,"""0xea9dfd24e2c5145d214f28e8da6a…","""0xbc2e4e52320a4b183428118cf371…",3,4,"""0xf091c1da488e63a711c13a508a07…","""0x296f55f8fb28e498b858d0bcda06…",314492516189394,"""314492516189394""",218342,606,"""0x2e15238c00000000000000000000…",2,623,547,2120913,208367,"""""",1,606,17849053713,3844,46930445531376,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686354775,105377999,"""0xea9dfd24e2c5145d214f28e8da6a…","""0x7fabc6ccce3d88f2ec357a6ff051…",3,5,"""0x1b3a60cf61fc60c1a111631af76b…","""0x00000000000000adc04c56bf30ac…",1200000000000000,"""1200000000000000""",195902,606,"""0x0000000000000000000000000000…",2,623,547,2261836,140923,"""""",1,606,17849053713,7668,93616716007958,0.684,null,null,null


In [7]:
def my_logic_for_address_model(_dataframes):
    return _dataframes["transactions"]

In [8]:
my_logic_for_address_model(dataframes)

dt,chain,network,chain_id,block_timestamp,block_number,block_hash,hash,nonce,transaction_index,from_address,to_address,value_64,value_lossless,gas,gas_price,input,transaction_type,max_fee_per_gas,max_priority_fee_per_gas,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_status,receipt_effective_gas_price,receipt_l1_gas_price,receipt_l1_gas_used,receipt_l1_fee,receipt_l1_fee_scalar,receipt_l1_blob_base_fee,receipt_l1_blob_base_fee_scalar,receipt_l1_base_fee_scalar
str,str,str,i32,u32,i64,str,str,i64,i64,str,str,i64,str,i64,i64,str,i32,i64,i64,i64,i64,str,i32,i64,i64,i64,i64,f64,i64,i64,i64
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x57f50c462841315a0e79a2650fd4…",140936,0,"""0xdeaddeaddeaddeaddeaddeaddead…","""0x4200000000000000000000000000…",0,"""0""",1000000,0,"""0x015d8eb900000000000000000000…",126,0,0,46913,46913,"""""",1,0,null,null,null,null,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x3e53770009212acc648e9faba440…",164300,1,"""0x593eb985684616112b09d6fb5d36…","""0x6a5ed50521eae16df75ad9dfb57a…",0,"""0""",5000000,214,"""0xa6f2ae3a""",0,0,0,217262,170349,"""""",1,214,17823396239,1884,22968226503764,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x484ed2a2299be2df85746aafd9eb…",966638,2,"""0xb8ff877ed78ba520ece21b1de784…","""0xa0cc33dd6f4819d473226257792a…",0,"""0""",173950,128,"""0x6c459a2800000000000000000000…",0,0,0,326160,108898,"""""",1,128,17823396239,6692,81583530659869,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0xe00836acfc2e456039312de9b217…",1,3,"""0x11ed66e8cd3fe0cd749d55664cd2…","""0xaaecb31961db53ea5ada85ee1bf4…",2496332147965529,"""2496332147965529""",22050,118,"""0x""",0,0,0,347160,21000,"""""",1,118,17823396239,1852,22578108006885,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686350777,105376000,"""0xf39de7bb1d732c9e4f44a936237a…","""0x88b6192a9ec8df00f92864badf46…",11,4,"""0xefb0eb8c4ecdff8d08b8fd5aacbc…","""0x52629961f71c1c2564c5aa22372c…",0,"""0""",339474,102,"""0x76319ee000000000000000000000…",2,122,49,676868,329708,"""""",1,102,17823396239,4856,59200481901423,0.684,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-06-09""","""op""","""mainnet""",10,1686354775,105377999,"""0xea9dfd24e2c5145d214f28e8da6a…","""0x64f9d972224d75e06c28e77b1f12…",15,3,"""0xd163a98169ac9fd8a5613a8f8504…","""0xb0d502e938ed5f4df2e681fe6e41…",0,"""0""",322651,606,"""0x87b21efc00000000000000000000…",2,623,547,1912546,301739,"""""",1,606,17849053713,2624,32035767188951,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686354775,105377999,"""0xea9dfd24e2c5145d214f28e8da6a…","""0xbc2e4e52320a4b183428118cf371…",3,4,"""0xf091c1da488e63a711c13a508a07…","""0x296f55f8fb28e498b858d0bcda06…",314492516189394,"""314492516189394""",218342,606,"""0x2e15238c00000000000000000000…",2,623,547,2120913,208367,"""""",1,606,17849053713,3844,46930445531376,0.684,null,null,null
"""2023-06-09""","""op""","""mainnet""",10,1686354775,105377999,"""0xea9dfd24e2c5145d214f28e8da6a…","""0x7fabc6ccce3d88f2ec357a6ff051…",3,5,"""0x1b3a60cf61fc60c1a111631af76b…","""0x00000000000000adc04c56bf30ac…",1200000000000000,"""1200000000000000""",195902,606,"""0x0000000000000000000000000000…",2,623,547,2261836,140923,"""""",1,606,17849053713,7668,93616716007958,0.684,null,null,null
